In [318]:
# Standard
import tempfile
import os
import sys
import pyparsing
from pyparsing import Word, alphas, Suppress, Combine, nums, string, Optional, Regex, ParseException
import pandas as pd
import time
import itertools

In [319]:
logfilecontent = '''
timestamp;call;arguments
1461864401;hello;world
1461864403;end;
1461864404;complex_operation;1,2,3,4
1461864407;multiply;1,2,3,4
1461864408;end;
1461864409;multiply;1,2,3,4
1461864410;end;
1461864411;end;
1461864415;multiply;5,6,7,8,9
1461864416;end;
1461864417;complex_operation;1,2,3,4
1461864421;add;1,2,3,4
1461864422;end;
1461864423;add;1,2,3,4
1461864424;end;
1461864425;end;
'''

In [320]:
timestamp = Word(nums)
call = Word(alphas + "_" + "-")
arguments = Word(alphas + nums + "," )

In [321]:
bnf = ( timestamp + ';' + call + ';' + Optional(arguments) )

In [322]:
def make_tempfile():
    fd, temp_file_name = tempfile.mkstemp()
    os.close(fd)
    f = open(temp_file_name,'wt')
    try:
        f.write(logfilecontent)
    finally:
        f.close()
    return temp_file_name

def cleanup(filename):
    os.unlink(filename)
        
filename = make_tempfile()

L=list()
with open(filename) as infile:
   for line in infile:
        try:
           results = bnf.parseString(line)
        except ParseException as pe:
            continue
        else:
           if (len(results) < 5 ):
            results.append('NULL')
           L.append(results)
           
print(L)
cleanup(filename)        

[(['1461864401', ';', 'hello', ';', 'world'], {}), (['1461864403', ';', 'end', ';', 'NULL'], {}), (['1461864404', ';', 'complex_operation', ';', '1,2,3,4'], {}), (['1461864407', ';', 'multiply', ';', '1,2,3,4'], {}), (['1461864408', ';', 'end', ';', 'NULL'], {}), (['1461864409', ';', 'multiply', ';', '1,2,3,4'], {}), (['1461864410', ';', 'end', ';', 'NULL'], {}), (['1461864411', ';', 'end', ';', 'NULL'], {}), (['1461864415', ';', 'multiply', ';', '5,6,7,8,9'], {}), (['1461864416', ';', 'end', ';', 'NULL'], {}), (['1461864417', ';', 'complex_operation', ';', '1,2,3,4'], {}), (['1461864421', ';', 'add', ';', '1,2,3,4'], {}), (['1461864422', ';', 'end', ';', 'NULL'], {}), (['1461864423', ';', 'add', ';', '1,2,3,4'], {}), (['1461864424', ';', 'end', ';', 'NULL'], {}), (['1461864425', ';', 'end', ';', 'NULL'], {})]


In [323]:
# Columns naming 
dframe = pd.DataFrame(L,columns=['timestamp','semicolon','call','semicolon','arguments'])

In [324]:
dframe


,timestamp,semicolon,call,semicolon,arguments
0,1461864401,;,hello,;,world
1,1461864403,;,end,;,NULL
2,1461864404,;,complex_operation,;,"1,2,3,4"
3,1461864407,;,multiply,;,"1,2,3,4"
4,1461864408,;,end,;,NULL
5,1461864409,;,multiply,;,"1,2,3,4"
6,1461864410,;,end,;,NULL
7,1461864411,;,end,;,NULL
8,1461864415,;,multiply,;,"5,6,7,8,9"
9,1461864416,;,end,;,NULL


In [325]:
del dframe['semicolon']

In [326]:
dframe

,timestamp,call,arguments
0,1461864401,hello,world
1,1461864403,end,NULL
2,1461864404,complex_operation,"1,2,3,4"
3,1461864407,multiply,"1,2,3,4"
4,1461864408,end,NULL
5,1461864409,multiply,"1,2,3,4"
6,1461864410,end,NULL
7,1461864411,end,NULL
8,1461864415,multiply,"5,6,7,8,9"
9,1461864416,end,NULL


In [327]:
dframe2 = dframe[::-1]
dframe2

,timestamp,call,arguments
15,1461864425,end,NULL
14,1461864424,end,NULL
13,1461864423,add,"1,2,3,4"
12,1461864422,end,NULL
11,1461864421,add,"1,2,3,4"
10,1461864417,complex_operation,"1,2,3,4"
9,1461864416,end,NULL
8,1461864415,multiply,"5,6,7,8,9"
7,1461864411,end,NULL
6,1461864410,end,NULL


In [328]:
C = []
dframe2_offset = len(dframe2.index)-2
complex_timestamp = 0

for id1, id2 in zip(dframe2.iterrows(),dframe2.ix[dframe2_offset:].iterrows()):
    print (id1[1]['timestamp'], id1[1]['call'])
    print (id2[1]['timestamp'], id2[1]['call'])
    print ('hello')
    
    if id1[1]['call'] == 'end' and id2[1]['call'] != 'end':
        C.append((id1[1]['timestamp'], id2[1]['call'], id2[1]['arguments'], id2[1]['timestamp']))
    elif  id1[1]['call'] == 'end' and id2[1]['call'] == 'end':
        complex_timestamp = id1[1]['timestamp']
    elif id1[1]['call'] != 'end' and id2[1]['call'] != 'end':
        C.append((complex_timestamp, id2[1]['call'], id2[1]['arguments'], id2[1]['timestamp']))
    


1461864425 end
1461864424 end
hello
1461864424 end
1461864423 add
hello
1461864423 add
1461864422 end
hello
1461864422 end
1461864421 add
hello
1461864421 add
1461864417 complex_operation
hello
1461864417 complex_operation
1461864416 end
hello
1461864416 end
1461864415 multiply
hello
1461864415 multiply
1461864411 end
hello
1461864411 end
1461864410 end
hello
1461864410 end
1461864409 multiply
hello
1461864409 multiply
1461864408 end
hello
1461864408 end
1461864407 multiply
hello
1461864407 multiply
1461864404 complex_operation
hello
1461864404 complex_operation
1461864403 end
hello
1461864403 end
1461864401 hello
hello


In [329]:
C

[('1461864424', 'add', '1,2,3,4', '1461864423'),
 ('1461864422', 'add', '1,2,3,4', '1461864421'),
 ('1461864425', 'complex_operation', '1,2,3,4', '1461864417'),
 ('1461864416', 'multiply', '5,6,7,8,9', '1461864415'),
 ('1461864410', 'multiply', '1,2,3,4', '1461864409'),
 ('1461864408', 'multiply', '1,2,3,4', '1461864407'),
 ('1461864411', 'complex_operation', '1,2,3,4', '1461864404'),
 ('1461864403', 'hello', 'world', '1461864401')]

In [330]:
dframe2 = pd.DataFrame(C,columns=['timestamp1','call','arguments','timestamp2'])

In [331]:
dframe2

,timestamp1,call,arguments,timestamp2
0,1461864424,add,"1,2,3,4",1461864423
1,1461864422,add,"1,2,3,4",1461864421
2,1461864425,complex_operation,"1,2,3,4",1461864417
3,1461864416,multiply,"5,6,7,8,9",1461864415
4,1461864410,multiply,"1,2,3,4",1461864409
5,1461864408,multiply,"1,2,3,4",1461864407
6,1461864411,complex_operation,"1,2,3,4",1461864404
7,1461864403,hello,world,1461864401


In [332]:
dframe2['timestamp1'] = dframe2['timestamp1'].astype(int)
dframe2['timestamp2'] = dframe2['timestamp2'].astype(int)
dframe2['timestamp1']= dframe2['timestamp1'] - dframe2['timestamp2']

In [333]:
dframe2

,timestamp1,call,arguments,timestamp2
0,1,add,"1,2,3,4",1461864423
1,1,add,"1,2,3,4",1461864421
2,8,complex_operation,"1,2,3,4",1461864417
3,1,multiply,"5,6,7,8,9",1461864415
4,1,multiply,"1,2,3,4",1461864409
5,1,multiply,"1,2,3,4",1461864407
6,7,complex_operation,"1,2,3,4",1461864404
7,2,hello,world,1461864401


In [334]:
del dframe2['timestamp2']

In [335]:
dframe2

,timestamp1,call,arguments
0,1,add,"1,2,3,4"
1,1,add,"1,2,3,4"
2,8,complex_operation,"1,2,3,4"
3,1,multiply,"5,6,7,8,9"
4,1,multiply,"1,2,3,4"
5,1,multiply,"1,2,3,4"
6,7,complex_operation,"1,2,3,4"
7,2,hello,world


In [336]:
dframe2.groupby(['call', 'arguments']).apply(lambda x: x['timestamp1'].sum())



call               arguments
add                1,2,3,4       2
complex_operation  1,2,3,4      15
hello              world         2
multiply           1,2,3,4       2
                   5,6,7,8,9     1
dtype: int64